In [49]:
DF_PATH       = "../data/processed/cleaned_data.pkl"
CLUSTERS_YAML_PATH = "../data/processed/features_skills_clusters_description.yaml"
ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageWorkedWith',    'DatabaseWorkedWith',    'WebframeWorkedWith',    'MiscTechWorkedWith']

## Import tools

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import yaml
import pickle
import random
import plotly 
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt


In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)

## Functions

In [4]:
def calculate_quality(ground_truth, prediction, metric_function, sort_values=False):
    quality_scores = {}
    for col in prediction.columns:
        role_pred  = prediction[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    return quality_scores


## Balance classes

In [5]:
roles_df = df.iloc[:, :16].copy()
role_sum = roles_df.sum(axis=0)
role_sum

Academic researcher                               581
Data or business analyst                          669
Data scientist or machine learning specialist     799
Database administrator                            296
DevOps specialist                                 677
Developer, QA or test                             493
Developer, back-end                              5503
Developer, desktop or enterprise applications    1671
Developer, embedded applications or devices       795
Developer, front-end                             2890
Developer, full-stack                            5578
Developer, game or graphics                       342
Developer, mobile                                1859
Engineer, data                                    483
Scientist                                         292
System administrator                              440
dtype: int64

In [6]:
# Resample roles
samples_per_class = 500
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0) 
    
    resampled_roles.append(sub_df)


In [7]:
# Construct dfs
roles_df  = pd.concat(resampled_roles)
df = df.loc[roles_df.index]

In [8]:
roles_df.sum(axis=0)

Academic researcher                               797
Data or business analyst                          767
Data scientist or machine learning specialist     871
Database administrator                            588
DevOps specialist                                 705
Developer, QA or test                             590
Developer, back-end                              1770
Developer, desktop or enterprise applications     892
Developer, embedded applications or devices       666
Developer, front-end                              893
Developer, full-stack                            1431
Developer, game or graphics                       560
Developer, mobile                                 800
Engineer, data                                    631
Scientist                                         642
System administrator                              647
dtype: int64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, 16:], df.iloc[:, :16], random_state=0)

## Random forest with non-linearity

In [10]:
rf_clf = make_pipeline(StandardScaler(), 
                       FeatureUnion([('linear_pca', PCA(n_components=40)), 
                                      ('kernel_pca', KernelPCA(n_components=40, 
                                                               kernel='rbf'))]),
                       RandomForestClassifier(random_state=0))

rf_clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('featureunion',
                 FeatureUnion(transformer_list=[('linear_pca',
                                                 PCA(n_components=40)),
                                                ('kernel_pca',
                                                 KernelPCA(kernel='rbf',
                                                           n_components=40))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=0))])

In [11]:
# Evaluate on train set
predictions =  pd.DataFrame(rf_clf.predict(X_train), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [12]:
train_scores

,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,98.87,96.94,91.36,94.07
Data or business analyst,99.37,97.44,96.12,96.77
Data scientist or machine learning specialist,99.27,97.26,96.09,96.67
Database administrator,99.30,94.68,95.96,95.31
DevOps specialist,99.57,99.22,95.83,97.50
"Developer, QA or test",99.42,99.00,92.76,95.78
"Developer, back-end",98.65,99.05,94.79,96.87
"Developer, desktop or enterprise applications",98.70,97.76,90.50,93.99
"Developer, embedded applications or devices",98.82,92.17,93.82,92.99
"Developer, front-end",99.47,98.52,96.80,97.65


## Predict sample entery

In [38]:
sample_skills = ['Pandas', 'TensorFlow', 'Torch/PyTorch', 'Python', 'Keras']

In [92]:
test_sample = pd.DataFrame(columns=X_train.columns)

In [93]:
test_sample.loc[0, sample_skills] = 1
test_sample.replace(np.nan, 0, inplace=True)

## Load the clusters dictionary

In [50]:
with open(CLUSTERS_YAML_PATH, "r") as stream:
    clusters_config = yaml.safe_load(stream)

In [51]:
molten_clusters = [(cluster_name, cluster_skill)
                   for cluster_name, cluster_skills in clusters_config.items()
                   for cluster_skill in cluster_skills]

clusters_df = pd.DataFrame(molten_clusters, columns=["cluster_name", "skill"])

In [54]:
sample_clusters = clusters_df.copy()

In [55]:
sample_clusters["sample_skills"] = sample_clusters["skill"].isin(sample_skills)

In [56]:
sample_clusters

,cluster_name,skill,sample_skills
0,skills_group_0,C#,False
1,skills_group_0,VBA,False
2,skills_group_0,Microsoft SQL Server,False
3,skills_group_0,ASP.NET,False
4,skills_group_0,ASP.NET Core,False
...,...,...,...
69,skills_group_9,Java,False
70,skills_group_9,Kotlin,False
71,skills_group_9,IBM DB2,False
72,skills_group_9,Oracle,False


In [91]:
cluster_features = sample_clusters.groupby("cluster_name")["sample_skills"].sum()
cluster_features

cluster_name
skills_group_0     0
skills_group_1     0
skills_group_10    0
skills_group_11    0
skills_group_12    0
skills_group_13    0
skills_group_14    0
skills_group_15    0
skills_group_16    0
skills_group_2     0
skills_group_3     0
skills_group_4     0
skills_group_5     1
skills_group_6     0
skills_group_7     4
skills_group_8     0
skills_group_9     0
Name: sample_skills, dtype: int64

In [103]:
for indx in cluster_features.index:
    test_sample[list(test_sample.filter(regex=indx))] = cluster_features[indx]

In [104]:
test_sample

,Assembly,Bash/Shell/PowerShell,C,C#,C++,Dart,Go,HTML/CSS,Haskell,Java,...,"(skills_clusters, skills_group_9)","(skills_clusters, skills_group_16)","(skills_clusters, skills_group_2)","(skills_clusters, skills_group_7)","(skills_clusters, skills_group_11)","(skills_clusters, skills_group_3)","(skills_clusters, skills_group_8)","(skills_clusters, skills_group_4)","(skills_clusters, skills_group_15)","(skills_clusters, skills_group_13)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0


In [110]:
prediction = rf_clf.predict_proba(test_sample)
positive_probs = [prob[0][1] for prob in prediction]
pd.Series(positive_probs, 
          index=Y_train.columns).sort_values(ascending=False)

Data scientist or machine learning specialist    0.9575
Scientist                                        0.1200
Academic researcher                              0.0700
Data or business analyst                         0.0625
Engineer, data                                   0.0100
Database administrator                           0.0000
DevOps specialist                                0.0000
Developer, QA or test                            0.0000
Developer, back-end                              0.0000
Developer, desktop or enterprise applications    0.0000
Developer, embedded applications or devices      0.0000
Developer, front-end                             0.0000
Developer, full-stack                            0.0000
Developer, game or graphics                      0.0000
Developer, mobile                                0.0000
System administrator                             0.0000
dtype: float64